# Label side

In [1]:
import numpy as np
from jesse import helpers, research

_, trading_1m = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-03-20"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

trading_1m.shape

(2743200, 6)

In [2]:
np.save("data/btc_1m.npy", trading_1m)

In [1]:
import numpy as np

trading_1m = np.load("data/btc_1m.npy")
trading_1m.shape

(2743200, 6)

In [2]:
from custom_indicators.toolbox.bar.dollar_bar import estimate_dollar_bar_threshold

for i in [45, 60 * 3, 60 * 12]:
    threshold = estimate_dollar_bar_threshold(trading_1m, i)
    print(f"{i}m: {threshold}")

45m: 380128379.094244
180m: 1578994802.314552
720m: 6432941784.13336


In [3]:
from jesse import helpers

from custom_indicators.toolbox.bar.dollar_bar import build_dollar_bar
from model.config import DOLLAR_BAR_THRESHOLD_MID

print(DOLLAR_BAR_THRESHOLD_MID)

tp = helpers.date_to_timestamp("2025-03-01")

label_bar = build_dollar_bar(trading_1m, DOLLAR_BAR_THRESHOLD_MID)

print(label_bar.shape)

1578994802.314552
(15335, 6)


In [4]:
from custom_indicators.toolbox.labeler.zigzag_labeler import zigzag

res = zigzag(label_bar[:, 2], 0.03)
np.unique(res, return_counts=True)

(array([-1,  1]), array([7350, 7985]))

In [5]:
label = np.column_stack((label_bar, res))
label

array([[1.57796202e+12, 7.15737000e+03, 7.28998000e+03, ...,
        6.86344000e+03, 2.21242464e+05, 1.00000000e+00],
       [1.57806666e+12, 7.29146000e+03, 7.43040000e+03, ...,
        7.25584000e+03, 2.14945802e+05, 1.00000000e+00],
       [1.57822008e+12, 7.43040000e+03, 7.76757000e+03, ...,
        7.30300000e+03, 2.10288562e+05, 1.00000000e+00],
       ...,
       [1.74241116e+12, 8.55055000e+04, 8.51798000e+04, ...,
        8.48776000e+04, 1.87567780e+04, 1.00000000e+00],
       [1.74241428e+12, 8.51798000e+04, 8.61776000e+04, ...,
        8.51639000e+04, 1.86524110e+04, 1.00000000e+00],
       [1.74242412e+12, 8.61777000e+04, 8.68098000e+04, ...,
        8.60514000e+04, 1.88527940e+04, 1.00000000e+00]])

In [6]:
np.save("data/label_side.npy", label[label[:, 0] <= tp])
print(label.shape)
print(label[label[:, 0] <= tp].shape)

(15335, 7)
(15059, 7)


In [7]:
from jesse import utils

df_dollar_bar = utils.numpy_candles_to_dataframe(
    label_bar[label_bar[:, 0] <= tp], name_date="time"
)
df_dollar_bar["zigzag"] = res[label_bar[:, 0] <= tp]
df_dollar_bar.head(1)

,time,open,close,high,low,volume,zigzag
2020-01-02 10:47:00,2020-01-02 10:47:00,7157.37,7289.98,7379.0,6863.44,221242.464,1


In [8]:
from lightweight_charts import Chart

chart = Chart(inner_width=1, inner_height=0.8)
chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_dollar_bar[["time", "open", "high", "low", "close", "volume"]])

line = chart2.create_line(name="zigzag", style="dotted")
line.set(df_dollar_bar[["time", "zigzag"]])

chart.show(block=True)

2025-03-25 17:35:31.922 python[63412:27800567] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-25 17:35:31.922 python[63412:27800567] +[IMKInputSession subclass]: chose IMKInputSession_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop

# Label meta

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

label_side_1h = np.load("data/label_side.npy")
print(label_side_1h.shape)
print(np.unique(label_side_1h[:, 6], return_counts=True))

df_features = pd.read_parquet("data/features_3h.parquet")
print(df_features.shape)

side_model = lgb.Booster(model_file="model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob
side_model_pred = np.where(side_model_pred_prob > 0.5, 1, -1)
print(np.unique(side_model_pred, return_counts=True))

(15059, 7)
(array([-1.,  1.]), array([7202, 7857]))
(15059, 11841)
(array([-1,  1]), array([7157, 7902]))


In [2]:
meta_label = (side_model_pred.astype(int) == label_side_1h[:, 6].astype(int)).astype(
    int
)
np.unique(meta_label, return_counts=True)

(array([0, 1]), array([ 2253, 12806]))

In [3]:
label_meta_1h = label_side_1h.copy()
label_meta_1h[:, 6] = meta_label
np.save("data/label_meta.npy", label_meta_1h)